In [71]:
from dotenv import find_dotenv,load_dotenv; _=load_dotenv(find_dotenv())

In [72]:
import json, os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from pydantic import BaseModel, Field
from typing import Union, Any, Optional
from enum import Enum
import googlemaps; gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

In [73]:
import requests
def get_current_location():
    geo_request_url = 'https://get.geojs.io/v1/ip/geo.json'
    data = requests.get(geo_request_url).json()
    return {
        'latitude': data['latitude'],
        'longitude': data['longitude'],
    }
print(json.dumps(get_current_location(),indent=2,ensure_ascii=False))

get_current_location_tool_description = """\
You can get current location with no input.
"""

get_current_location_tool = StructuredTool.from_function(
    func = get_current_location,
    # name='CurrentLocationGetter',
    description=get_current_location_tool_description,
)

{
  "latitude": "35.1796",
  "longitude": "136.9166"
}


In [74]:
def simplified_place_dict(res):
    r = res[0]
    return {
        'address': r['formatted_address'],
        'location': r['geometry']['location'],
        'types': r['types']
    }

def find_place_with_text_func(query: str):
    places_result = gmaps.find_place(query,input_type='textquery')
    geocode_results = [
        gmaps.geocode(place_id=r['place_id'])
        for r in places_result['candidates']
    ]
    # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
    return [simplified_place_dict(r) for r in geocode_results]
# print(find_place_with_text_func(query='ガスト港東通店'))
find_place_tool_description = """\
A Find Place request takes a text input, and returns a place.
The text input can be any kind of Places data, for example,
a name, address, or phone number.
"""

find_place_tool = Tool.from_function(
    func = find_place_with_text_func,
    name='PlaceSearch',
    description=find_place_tool_description,
)

In [85]:
class Mode(Enum):
    DRIVING = "driving"
    WALKING = "walking"
    BYCYCLING = "bicycling"
    TRANSIT = "transit"

class DirectionsInput(BaseModel):
    # origin: Union[str,Any] = Field(
    origin: str = Field(
        description='The address or latitude/longitude value from which you wish to calculate directions.'
    )
    # destination: Union[str,Any] = Field(
    destination: str = Field(
        description='The address or latitude/longitude value from which you wish to calculate directions.'
    )
    mode: Mode = Field(
        description='Specifies the mode of transport to use when calculating directions.'
    )
    # departure_time: Optional[int] = Field(
    #     description='Specifies the desired time of departure. Note: you can't specify both departure_time and arrival_time.'
    # )
    # arrival_time: Optional[int] = Field(
    #     description='Specifies the desired time of arrival for transit directions. Note: you can't specify both departure_time and arrival_time.'
    # )


def simplified_directions_dict(res):
    r = res['legs'][0]
    return {
        'arrival_time': r['arrival_time']['text'],
        'departure_time': r['departure_time']['text'],
        'distance': r['distance']['text'],
        'duration': r['duration']['text'],
    }

def get_direction_func(params: DirectionsInput):
    places_result = gmaps.directions_result = gmaps.directions(
        origin=params.origin,
        destination=params.destination,
        mode=params.mode.value,
    )
    # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
    return [simplified_directions_dict(r) for r in places_result]
# print(find_place_with_text_func(
#     query=DirectionsInput(
#     origin='エスターテ南青山',
#     destination='ガスト港東通店',
#     mode='walking')
# ))
directions_tool_description = """\
Get directions between an origin point and a destination point.
"""

get_direction_tool = Tool.from_function(
    func = get_direction_func,
    name='Directions',
    description=directions_tool_description,
)

ValidationError: 1 validation error for Tool
input_schema
  extra fields not permitted (type=value_error.extra)

In [76]:
llm = ChatOpenAI(temperature=0)

In [77]:
tools = [
    get_current_location_tool,
    find_place_tool,
    get_direction_tool
]

In [78]:
agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

In [79]:
input_text = '近くのガストを教えて'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `get_current_location` with `{}`


{'latitude': '35.1796', 'longitude': '136.9166'}
Invoking: `PlaceSearch` with `ガスト`


[{'address': 'Japan, 〒457-0834 愛知県名古屋市南区港東通１丁目２４', 'location': {'lat': 35.091708, 'lng': 136.916568}, 'types': ['establishment', 'food', 'meal_delivery', 'meal_takeaway', 'point_of_interest', 'restaurant']}]近くにあるガストは、愛知県名古屋市南区港東通１丁目２４にあります。

> Finished chain.


'近くにあるガストは、愛知県名古屋市南区港東通１丁目２４にあります。'

In [80]:
input_text = '近くのガストまで徒歩で何分？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `get_current_location` with `{}`


{'latitude': '35.1796', 'longitude': '136.9166'}
Invoking: `PlaceSearch` with `ガスト`


[{'address': 'Japan, 〒457-0834 愛知県名古屋市南区港東通１丁目２４', 'location': {'lat': 35.091708, 'lng': 136.916568}, 'types': ['establishment', 'food', 'meal_delivery', 'meal_takeaway', 'point_of_interest', 'restaurant']}]
Invoking: `Directions` with `35.1796,136.9166|35.091708,136.916568`




AttributeError: 'str' object has no attribute 'origin'

In [84]:
from pydantic import BaseModel

class MyToolInput(BaseModel):
    var1: str
    var2: int
    var3: float

def my_tool_func(input: MyToolInput):
    '''
    You should always use this func.
    '''
    # Do something with the input
    return 'complete'

from langchain.tools import StructuredTool

my_tool = StructuredTool.from_function(
    name="My Tool", 
    func=my_tool_func,
    input_schema=MyToolInput
)

agent = initialize_agent(
    [my_tool], llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

ValidationError: 1 validation error for StructuredTool
input_schema
  extra fields not permitted (type=value_error.extra)